In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from uszipcode import SearchEngine

In [3]:
zori = pd.read_csv('Zillow_Data_direct/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv', index_col=0)
zri = pd.read_csv('Zip_Zri_AllHomesPlusMultifamily.csv', index_col=0)

zri_info = zri[['RegionName', 'CountyName', 'City', 'State', 'Metro']]

### Zori EDA

In [4]:
print(zori.shape)
#zori.columns
#zori.head()

(2726, 92)


#### Looking at nulls by Month

In [5]:
zori_nulls = zori.isnull().sum(axis=0)
#month_nulls = month_nulls.isnull().sum(axis=1)
zori_nulls = zori_nulls[zori_nulls > 0].sort_values(ascending= False)
zori_nulls = pd.DataFrame(zori_nulls, columns = ['Nulls'])
zori_nulls['Yr_Mo'] = zori_nulls.index
zori_nulls.head()

,Nulls,Yr_Mo
2014-01,726,2014-01
2014-03,639,2014-03
2014-02,626,2014-02
2014-04,404,2014-04
2014-11,346,2014-11


In [8]:
zori_nulls = zori_nulls.merge(zori_nulls.Yr_Mo.apply(lambda x: pd.Series({"Year": int(x[0:4]), "Month": int(x[5:])})),
                 left_index=True, right_index=True)
zori_nulls['PercentNull'] = np.round(zori_nulls['Nulls']/zori.shape[0], 2)#.describe()
zori_nulls.drop('Yr_Mo', axis=1, inplace=True)
zori_nulls.head()

,Nulls,Year,Month,PercentNull
2014-01,726,2014,1,0.27
2014-03,639,2014,3,0.23
2014-02,626,2014,2,0.23
2014-04,404,2014,4,0.15
2014-11,346,2014,11,0.13


In [9]:
zori_year_nulls = zori_nulls.groupby(['Year']).agg({'Nulls': np.sum, 'PercentNull': np.mean}).reset_index()
zori_year_nulls

,Year,Nulls,PercentNull
0,2014,4488,0.138333
1,2015,1670,0.050833
2,2016,785,0.023333
3,2017,320,0.010833
4,2018,226,0.009167
5,2019,178,0.005833
6,2020,287,0.010000
7,2021,252,0.018000


In [10]:
month_nulls = zori_nulls.pivot(index='Year',columns='Month',values = 'Nulls').reset_index().rename_axis(None, axis=1)

months = list(month_nulls)
months.remove("Year")

month_nulls['Yr_Sum']= month_nulls[months].sum(axis=1)
month_nulls

,Year,1,2,3,4,5,6,7,8,9,10,11,12,Yr_Sum
0,2014,726.0,626.0,639.0,404.0,309.0,236.0,184.0,193.0,213.0,269.0,346.0,343.0,4488.0
1,2015,251.0,264.0,164.0,146.0,108.0,90.0,53.0,73.0,89.0,168.0,99.0,165.0,1670.0
2,2016,117.0,89.0,81.0,69.0,37.0,34.0,49.0,38.0,45.0,91.0,49.0,86.0,785.0
3,2017,47.0,46.0,26.0,23.0,19.0,13.0,16.0,10.0,18.0,30.0,24.0,48.0,320.0
4,2018,23.0,23.0,18.0,27.0,14.0,13.0,16.0,15.0,14.0,15.0,23.0,25.0,226.0
5,2019,20.0,12.0,11.0,15.0,11.0,11.0,11.0,15.0,17.0,16.0,24.0,15.0,178.0
6,2020,13.0,14.0,16.0,24.0,20.0,16.0,18.0,19.0,19.0,30.0,40.0,58.0,287.0
7,2021,37.0,70.0,63.0,29.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0


### Finding Outliers

#### Convert to long format

In [17]:
zori = pd.read_csv('Zillow_Data_direct/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv', index_col=0)
zori.head()
zori.shape

(2726, 92)

In [18]:
# Tried using uszipcode library. Instead, will use data from the ZRI dataset to find zip codes' county name

# search = SearchEngine(simple_zipcode=True)
# zori['County'] = zori['RegionName'].apply(lambda x: search.by_zipcode(x).to_dict().get('county'))
# zori['State'] = zori['RegionName'].apply(lambda x: search.by_zipcode(x).to_dict().get('state'))
# zori['Pop'] = zori['RegionName'].apply(lambda x: search.by_zipcode(x).to_dict().get('population'))
# zori.head()

In [19]:
print(zori.shape)
zori = pd.merge(zori, zri_info, how='left', on='RegionName')
print(zori.shape)

zori.head()

(2726, 92)
(2726, 96)


,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,CountyName,City,State,Metro
0,10025,1,"New York, NY",2883.0,2895.0,2907.0,2920.0,2933.0,2945.0,2958.0,...,2812.0,2784.0,2758.0,2732.0,2706.0,2682.0,New York County,New York,NY,New York-Newark-Jersey City
1,60657,2,"Chicago, IL",1484.0,1489.0,1495.0,1500.0,1505.0,1510.0,1515.0,...,1670.0,1665.0,1661.0,1657.0,1653.0,1649.0,Cook County,Chicago,IL,Chicago-Naperville-Elgin
2,10023,3,"New York, NY",3134.0,3143.0,3152.0,3161.0,3169.0,3178.0,3186.0,...,2981.0,2947.0,2914.0,2882.0,2849.0,2819.0,New York County,New York,NY,New York-Newark-Jersey City
3,77494,4,"Houston, TX",1523.0,1526.0,1529.0,1532.0,1534.0,1537.0,1539.0,...,1578.0,1584.0,1590.0,1597.0,1603.0,1610.0,Harris County,Katy,TX,Houston-The Woodlands-Sugar Land
4,60614,5,"Chicago, IL",1679.0,1684.0,1689.0,1694.0,1698.0,1703.0,1707.0,...,1873.0,1865.0,1856.0,1848.0,1839.0,1831.0,Cook County,Chicago,IL,Chicago-Naperville-Elgin


In [20]:
info_cols = ['SizeRank', 'MsaName', 'CountyName', 'City', 'State', 'Metro']

print(zori.shape)
zori_df = zori.copy()
zori_df.drop(info_cols, axis=1, inplace=True)
zori_df.head()
print(zori_df.shape)

(2726, 96)
(2726, 90)


In [51]:
zori_melt = pd.melt(zori_df, id_vars=['RegionName'], var_name='Yr_Mo', value_name='RI')
zori_melt.head()

#zori_melt.shape

,RegionName,Yr_Mo,RI
0,10025,2014-01,2883.0
1,60657,2014-01,1484.0
2,10023,2014-01,3134.0
3,77494,2014-01,1523.0
4,60614,2014-01,1679.0


In [70]:
zori_long = zori_melt.merge(zori_melt.Yr_Mo.apply(lambda x: pd.Series({"Year": int(x[0:4]), "Month": int(x[5:])})),
                 left_index=True, right_index=True)
zori_long.head()

,RegionName,Yr_Mo,RI,Year,Month
0,10025,2014-01,2883.0,2014,1
1,60657,2014-01,1484.0,2014,1
2,10023,2014-01,3134.0,2014,1
3,77494,2014-01,1523.0,2014,1
4,60614,2014-01,1679.0,2014,1


In [71]:
zori_long = pd.merge(zori_long, zori[['CountyName', 'RegionName']], how='outer', on='RegionName')

In [72]:
zori_long.head()

,RegionName,Yr_Mo,RI,Year,Month,CountyName
0,10025,2014-01,2883.0,2014,1,New York County
1,10025,2014-02,2895.0,2014,2,New York County
2,10025,2014-03,2907.0,2014,3,New York County
3,10025,2014-04,2920.0,2014,4,New York County
4,10025,2014-05,2933.0,2014,5,New York County


#### Interpolating Missing Values

In [43]:
#zori_null = zori_long[zori_long['RI'].isnull()]


In [73]:
#filled_df = zori_long.groupby(['RegionName', 'Year'])['RI'].transform(pd.DataFrame.interpolate())
zori_save = zori_long.copy()

for i in range(len(zori_long.RegionName.unique())):
    mask = zori_long.loc[:,'RegionName']==zori_long.RegionName.unique()[i]
    zori_long[mask]= zori_long[mask].interpolate(method='linear')
    
zori_long[zori_long['Year']==2014]

,RegionName,Yr_Mo,RI,Year,Month,CountyName
0,10025,2014-01,2883.000000,2014,1,New York County
1,10025,2014-02,2895.000000,2014,2,New York County
2,10025,2014-03,2907.000000,2014,3,New York County
3,10025,2014-04,2920.000000,2014,4,New York County
4,10025,2014-05,2933.000000,2014,5,New York County
5,10025,2014-06,2945.000000,2014,6,New York County
6,10025,2014-07,2958.000000,2014,7,New York County
7,10025,2014-08,2970.000000,2014,8,New York County
8,10025,2014-09,2983.000000,2014,9,New York County
9,10025,2014-10,2995.000000,2014,10,New York County


In [74]:
for i in range(len(zori_long.RegionName.unique())):
    mask = zori_long.loc[:,'RegionName']==zori_long.RegionName.unique()[i]
    zori_long[mask]= zori_long[mask].interpolate(method='linear', limit_direction = 'backward')
zori_long[zori_long['Year']==2014]

,RegionName,Yr_Mo,RI,Year,Month,CountyName
0,10025,2014-01,2883.000000,2014,1,New York County
1,10025,2014-02,2895.000000,2014,2,New York County
2,10025,2014-03,2907.000000,2014,3,New York County
3,10025,2014-04,2920.000000,2014,4,New York County
4,10025,2014-05,2933.000000,2014,5,New York County
5,10025,2014-06,2945.000000,2014,6,New York County
6,10025,2014-07,2958.000000,2014,7,New York County
7,10025,2014-08,2970.000000,2014,8,New York County
8,10025,2014-09,2983.000000,2014,9,New York County
9,10025,2014-10,2995.000000,2014,10,New York County


In [57]:
fill_nulls = zori_long.isnull().sum(axis=0)
#month_nulls = month_nulls.isnull().sum(axis=1)
fill_nulls.head()

RegionName       0
Yr_Mo            0
RI            1403
Year             0
Month            0
dtype: int64

In [75]:
zori_long['zip_mean'] = zori_long.groupby('RegionName')['RI'].transform('mean')

In [87]:
zori_long['zip_std'] = zori_long.groupby('RegionName')['RI'].transform('std')
zori_long.head()

,RegionName,Yr_Mo,RI,Year,Month,CountyName,zip_mean,zip_std,Outlier_High,Outlier_Low
0,10025,2014-01,2883.0,2014,1,New York County,2951.25,112.279389,False,False
1,10025,2014-02,2895.0,2014,2,New York County,2951.25,112.279389,False,False
2,10025,2014-03,2907.0,2014,3,New York County,2951.25,112.279389,False,False
3,10025,2014-04,2920.0,2014,4,New York County,2951.25,112.279389,False,False
4,10025,2014-05,2933.0,2014,5,New York County,2951.25,112.279389,False,False


In [88]:
zori_long['Outlier_High'] = zori_long['RI']>(zori_long['zip_mean']+(2*zori_long['zip_std']))
zori_long['Outlier_Low'] = zori_long['RI']<(zori_long['zip_mean']-(2*zori_long['zip_std']))
zori_long.head()

,RegionName,Yr_Mo,RI,Year,Month,CountyName,zip_mean,zip_std,Outlier_High,Outlier_Low
0,10025,2014-01,2883.0,2014,1,New York County,2951.25,112.279389,False,False
1,10025,2014-02,2895.0,2014,2,New York County,2951.25,112.279389,False,False
2,10025,2014-03,2907.0,2014,3,New York County,2951.25,112.279389,False,False
3,10025,2014-04,2920.0,2014,4,New York County,2951.25,112.279389,False,False
4,10025,2014-05,2933.0,2014,5,New York County,2951.25,112.279389,False,False


In [89]:
zori_long[zori_long['Outlier_High']==1]

,RegionName,Yr_Mo,RI,Year,Month,CountyName,zip_mean,zip_std,Outlier_High,Outlier_Low
5528,77077,2014-11,1284.0,2014,11,Harris County,1224.833333,28.983630,True,False
5529,77077,2014-12,1291.0,2014,12,Harris County,1224.833333,28.983630,True,False
23417,77007,2014-11,1762.0,2014,11,Harris County,1600.416667,75.461144,True,False
23418,77007,2014-12,1781.0,2014,12,Harris County,1600.416667,75.461144,True,False
53945,21207,2014-12,1349.0,2014,12,Baltimore County,1250.041667,47.396546,True,False
89543,50266,2014-10,1253.0,2014,10,Polk County,1196.500000,27.428270,True,False
89544,50266,2014-11,1258.0,2014,11,Polk County,1196.500000,27.428270,True,False
89545,50266,2014-12,1263.0,2014,12,Polk County,1196.500000,27.428270,True,False
132443,77025,2014-12,1551.0,2014,12,Harris County,1354.625000,83.410218,True,False
211358,21076,2020-01,2084.0,2020,1,Anne Arundel County,2036.833333,23.023339,True,False


In [90]:
zori_long[zori_long['Outlier_Low']==1]

,RegionName,Yr_Mo,RI,Year,Month,CountyName,zip_mean,zip_std,Outlier_High,Outlier_Low
23407,77007,2014-01,1381.0,2014,1,Harris County,1600.416667,75.461144,False,True
23408,77007,2014-02,1424.0,2014,2,Harris County,1600.416667,75.461144,False,True
113297,63119,2014-01,1179.0,2014,1,Saint Louis County,1223.416667,20.332470,False,True
157441,55443,2014-01,1646.0,2014,1,Hennepin County,1820.958333,82.331160,False,True
157442,55443,2014-02,1646.0,2014,2,Hennepin County,1820.958333,82.331160,False,True
168121,77041,2014-01,1591.0,2014,1,Harris County,1667.583333,38.114526,False,True


In [91]:
zori_long.shape

(242614, 10)

In [92]:
zori_long.to_csv('mostly_clean_zori.csv', index=False)

In [94]:
zori_save[(zori_save['RegionName']==50266) & (zori_save['Year']==2014)]

,RegionName,Yr_Mo,RI,Year,Month,CountyName
89534,50266,2014-01,NaN,2014,1,Polk County
89535,50266,2014-02,NaN,2014,2,Polk County
89536,50266,2014-03,NaN,2014,3,Polk County
89537,50266,2014-04,1145.0,2014,4,Polk County
89538,50266,2014-05,1164.0,2014,5,Polk County
89539,50266,2014-06,1183.0,2014,6,Polk County
89540,50266,2014-07,1202.0,2014,7,Polk County
89541,50266,2014-08,1219.0,2014,8,Polk County
89542,50266,2014-09,1236.0,2014,9,Polk County
89543,50266,2014-10,1253.0,2014,10,Polk County


In [95]:
zori_long[(zori_long['RegionName']==50266) & (zori_long['Year']==2014)]

,RegionName,Yr_Mo,RI,Year,Month,CountyName,zip_mean,zip_std,Outlier_High,Outlier_Low
89534,50266,2014-01,1145.0,2014,1,Polk County,1196.5,27.42827,False,False
89535,50266,2014-02,1145.0,2014,2,Polk County,1196.5,27.42827,False,False
89536,50266,2014-03,1145.0,2014,3,Polk County,1196.5,27.42827,False,False
89537,50266,2014-04,1145.0,2014,4,Polk County,1196.5,27.42827,False,False
89538,50266,2014-05,1164.0,2014,5,Polk County,1196.5,27.42827,False,False
89539,50266,2014-06,1183.0,2014,6,Polk County,1196.5,27.42827,False,False
89540,50266,2014-07,1202.0,2014,7,Polk County,1196.5,27.42827,False,False
89541,50266,2014-08,1219.0,2014,8,Polk County,1196.5,27.42827,False,False
89542,50266,2014-09,1236.0,2014,9,Polk County,1196.5,27.42827,False,False
89543,50266,2014-10,1253.0,2014,10,Polk County,1196.5,27.42827,True,False
